In [1]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [2]:
from deeplifting.models import DeepliftingSkipMLP
from deeplifting.optimization import set_seed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt

In [37]:
input_size = 1
hidden_sizes = (256,) * 3
dimensions = 2
bounds = [(0, 14)] * 2
activation = 'leaky_relu'
output_activation = 'sine'
agg_function = 'average'

# How difference are the initial seeds?
for i in range(20):
    model = DeepliftingSkipMLP(
        input_size=input_size,
        hidden_sizes=hidden_sizes,
        output_size=dimensions,
        bounds=bounds,
        skip_every_n=2,
        activation=activation,
        output_activation=output_activation,
        agg_function=agg_function,
        seed=i,
    )

    outputs = model(None)
    outputs = outputs.mean(axis=0)
    print(outputs.flatten())

tensor([11.4785,  7.1925], grad_fn=<MeanBackward1>)
tensor([ 7.1474, 13.8895], grad_fn=<MeanBackward1>)
tensor([13.8010,  6.0762], grad_fn=<MeanBackward1>)
tensor([13.2258, 11.4424], grad_fn=<MeanBackward1>)
tensor([2.6649, 6.7658], grad_fn=<MeanBackward1>)
tensor([9.7807, 0.5950], grad_fn=<MeanBackward1>)
tensor([7.3448, 8.3850], grad_fn=<MeanBackward1>)
tensor([ 0.0819, 12.4842], grad_fn=<MeanBackward1>)
tensor([ 0.0435, 10.4527], grad_fn=<MeanBackward1>)
tensor([13.6776,  9.0483], grad_fn=<MeanBackward1>)
tensor([10.2468,  4.4712], grad_fn=<MeanBackward1>)
tensor([10.8153, 13.8738], grad_fn=<MeanBackward1>)
tensor([ 0.1759, 13.9828], grad_fn=<MeanBackward1>)
tensor([13.6755,  3.9772], grad_fn=<MeanBackward1>)
tensor([9.3147e+00, 4.1723e-07], grad_fn=<MeanBackward1>)
tensor([ 3.6321, 12.8621], grad_fn=<MeanBackward1>)
tensor([13.9935,  6.7280], grad_fn=<MeanBackward1>)
tensor([6.7727, 0.0526], grad_fn=<MeanBackward1>)
tensor([ 8.2983, 13.0774], grad_fn=<MeanBackward1>)
tensor([4.1377